# Major World Indices Overview

In [1]:
# import 
import dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
import pandas as pd
#import pandas_datareader as data
import pandas_datareader as pdr
from pandas_datareader import wb
from datetime import datetime as dt
import yfinance as yf
import numpy as np
import date_converter
%config InlineBackend.figure_format = 'retina'
global periods
periods = ''

print(f"pandas  version = {pdr.__version__}")
fmt = "%d-%m-%Y - %H:%M"
todays = pd.datetime.today()
print(todays.strftime(fmt))

pandas  version = 0.7.0
22-10-2019 - 23:59


In [2]:
# ดึงรายชื่อ ticker
global list1,list2,list3

tick = pd.read_excel('data/world_market_symbols.xlsx',header=0)  
list1 = list(tick['shortname'])
list2 = list(tick['symbol'])
list3 = list(tick['country_code'])

def get_tickers(value = None,data = 'shortname'): 
    global list1,list2,list3
    if data == 'shortname':
        tickers = dict( zip( list1, list2))
        return tickers 
    elif data == 'map_shortname':
        tickers = dict( zip( list2, list1))    
        ticker = tickers[value]    
        return ticker
    else :
        countrys = dict( zip( list2, list3))    
        country = countrys[value]  
        if data == 'map_country':
            return country
        else : 
            co = pd.read_csv('data/country.csv' )
            list4 = list(co['iso2c'])
            list5 = list(co['name'])
            names = dict( zip( list4, list5)) 
            
            country_name = names[country.upper()]
            print('country_name')
            return country_name

def fmtn(number):
    """
    Formatting helper - float
    """
    if np.isnan(number):
        return '-'
    return format(number, ',.2f')

def date_time(times):
    timestamp = times
    dt_object = dt.fromtimestamp(timestamp)
    print(dt_object)
    return dt_object



In [3]:
## NavBar

PLOTLY_LOGO = "https://gitlab.com/yosarawut/financial-investment/raw/master/img/planet-earth.png?inline=false"

tickers = get_tickers(data = 'shortname')
dropdown = dbc.Row(
    [
        dbc.Col(dcc.Dropdown(
            id='my-dropdown',
            options=[{'label': s[0], 'value': str(s[1])}
                     for s in zip(tickers, tickers.values())],
            value='^GSPC',style={"margin-top": "10px", "margin-left": "10px", 
                                 "margin-right": "10px", "width": "300px"}))],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",align="center")
periods = '1y'
navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                    dbc.Col(dbc.NavbarBrand(
                        "Major World Indices Overview", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://dash.plot.ly",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(dropdown, id="navbar-collapse", navbar=True),
    ],
    color="primary",
    dark=True,
    className="mt-4",
)

In [4]:
graph =    dcc.Graph(id='my-graph', 
    style={"margin-left": "7%","margin-right": "auto","margin-top": "10px","width": "85%",'height':'350px'})

In [ ]:
row_1 = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(html.H2(id='ticker_name',className="display-5",style={"font-weight": "bold",'color':'#2A4B7C',
                                                                              "margin-left": "10px",}), md=6),
                dbc.Col(html.Div(""), md=2),
                dbc.Col(html.H2(id='regularMarketPrice',style={"text-align": "right",'color':'#2A4B7C',"font-weight": "bold"}), md=3),
                dbc.Col(html.Div(id='currency',style={"text-align": "left","margin-top": "18px",'color':'gray'}), md=1),
            ],
        style={"margin-top": "10px","margin-left": "10px","margin-right": "10px"}),
        dbc.Row(
            [
                dbc.Col(html.Img(id = 'flag', src='', style={"margin-left": "60%","margin-right": "auto",
                                                             'background-color': '#2A4B7C'}),md=1),
      
                dbc.Col(html.Div(id='country-name',style={"text-align": "left",'font-size': '18px',"font-weight": "bold"}),md=2),
                dbc.Col(html.Div(id='lastdate',style={"text-align": "left",'color':'gray'}),md=2),                
                dbc.Col(html.Div(id="marketState",
                                 style={"text-align": "right","margin-left": "10px", "font-weight": "normal",
                                        'color':'gray','font-size': '14px'}), md=2),
                dbc.Col(html.Div(""), md=1),
                dbc.Col(html.H4(id='pricechange',style={"text-align": "right","font-weight": "bold"}), md=2),
                dbc.Col(html.H4(id='percentchange',style={"text-align": "left","font-weight": "bold"}), md=2),
            ],
            style={"margin-top": "5px","margin-left": "10px","margin-right": "10px"}
        ),
    ]
)

In [ ]:
row_2 = html.Div(
    [
        dbc.Row(
            [
                
                dbc.Col(html.Div(id='prev-close',
                                 style={"text-align": "center"}), md=1),
                dbc.Col(html.Div(id="reg-open",
                                 style={"text-align": "center"}), md=1),
                dbc.Col(html.Div(id="reg-volumn",
                                 style={"text-align": "center"}), md=2),
                dbc.Col(html.Div(id="day-range",
                                 style={"text-align": "center"}), md=2),
                dbc.Col(html.Div(id="year-range",
                                 style={"text-align": "center"}), md=2),
                dbc.Col(html.Div(""), md=2),
                
                           
         
                        
            ],
            style={"margin-top": "5px", "margin-left": "10px", "margin-right": "10px", "font-weight": "bold",
                   'font-size': '18px', 'color': '#2A4B7C'}
        ),

        dbc.Row(
            [
                
                dbc.Col(html.Div("Prev Close", style={
                        "text-align": "center"}), md=1),
                dbc.Col(html.Div("Open", style={
                        "text-align": "center"}), md=1),
                dbc.Col(html.Div("Volumn", style={
                        "text-align": "center"}), md=2),
                dbc.Col(html.Div("Day's Range", style={
                        "text-align": "center"}), md=2),
                dbc.Col(html.Div("52 Week's Range", style={
                        "text-align": "center"}), md=2),
                daq.Slider(id="slider", min=1, max=100, step=0.5, value=50,color='#FF6F61'),

            ],
            style={"margin-top": "5px", "margin-left": "10px",
                   "margin-right": "10px", 'color': 'gray', 'font-size': '14px'}
        )])   


In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.layout = html.Div([navbar, row_1,row_2,graph])





In [ ]:
@app.callback([Output('ticker_name', 'children'),Output('flag', 'src')],
               [Input('my-dropdown', 'value')])
def update_output_children(selected_dropdown_value): 
    country_code = get_tickers(selected_dropdown_value,'map_country')
    flag_file = 'assets/flags/4x3/'+ (f'{country_code}')+'.svg'
    
    return get_tickers(selected_dropdown_value,data = 'map_shortname'),flag_file

In [ ]:
def get_info(ti):
    sym = yf.Ticker(ti)
    rep = sym.info
    return rep

In [ ]:
@app.callback([Output('regularMarketPrice', 'children'),Output('lastdate', 'children'),
                              
                      Output('country-name', 'children'),Output('marketState', 'children')],
                      [Input('my-dropdown', 'value')])
def update_output_children(selected_dropdown_value):      
    a = get_info(selected_dropdown_value)
    times = a['regularMarketTime']
    reg_time = date_time(times)
    
    country_name = get_tickers(selected_dropdown_value,data = 'country_name')   
    
    market_state = 'Market State :  ' + a['marketState']
    return fmtn(a['regularMarketPrice']),reg_time.strftime(fmt),country_name,market_state

In [ ]:
@app.callback([Output('currency', 'children'), Output('prev-close', 'children'),
               Output('reg-open', 'children'), Output('reg-volumn', 'children'),
               Output('year-range', 'children'),
               Output('day-range', 'children')],[Input('my-dropdown', 'value')])
def update_output_children(selected_dropdown_value):
    a = get_info(selected_dropdown_value)
    vol = a['regularMarketVolume']
    if float(vol) > 999999:
        reg_volumn = fmtn((vol/1000000)) + " M"
    elif float(vol) > 9999:
        reg_volumn = fmtn((vol/1000)) + " K"
    else:
        reg_volumn = 'N/A'
        
    year_ranges = fmtn(a['fiftyTwoWeekLow']) + ' - ' + \
        fmtn(a['fiftyTwoWeekHigh'])

    day_ranges = fmtn(a['regularMarketDayLow']) + ' - ' + \
        fmtn(a['regularMarketDayHigh'])

    return a['currency'], fmtn(a['regularMarketPreviousClose']), fmtn(a['regularMarketOpen']), reg_volumn,year_ranges, day_ranges

In [ ]:
@app.callback([Output('slider', 'min'),Output('slider', 'max'),
              Output('slider', 'value')],[Input('my-dropdown', 'value')])
def update_slider_value(selected_dropdown_value):
               a = get_info(selected_dropdown_value) 
               
               return a['fiftyTwoWeekLow'],a['fiftyTwoWeekHigh'],a['regularMarketPrice']   
    

In [ ]:
@app.callback([Output('pricechange', 'children'),Output('percentchange', 'children')],
               [Input('my-dropdown', 'value')])
def update_output_children(selected_dropdown_value):   
    a = get_info(selected_dropdown_value) 
    price_chg = fmtn(a['regularMarketChange'])
    percent_chg = fmtn(a['regularMarketChangePercent'])    
    if float(price_chg) < 0 :
        pric = html.Div(price_chg, style={'color': 'red'})
        perc= html.Div('( '+percent_chg+'% )', style={'color': 'red'})
    else:
        pric= html.Div('+ '+price_chg, style={'color': '#2A4B7C'})   
        perc= html.Div('(+ '+percent_chg+'% )', style={'color': '#2A4B7C'})  
  
    return pric,perc

In [ ]:
@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    
    periods = '1y'    
    data = yf.Ticker(selected_dropdown_value)    
    df = data.history(period=periods)          
   
    return {
        'data': [{
            'x': df.index,
            'y': df.Close,            
        }],
        'layout': {'margin': {'l': 30, 'r': 0, 't': 20, 'b': 30}}       
    }

In [ ]:
app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\dash\resources.py:44: UserWarning:

A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available

127.0.0.1 - - [22/Oct/2019 23:59:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:43] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1565712429 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:43] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1565712429 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:43] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1565712429 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:43] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1565712429 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:43] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap

2019-10-22 23:59:42
country_name


127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:59:54] "POST /_dash-update-component HTTP/1.1" 200 -


2019-10-22 17:02:42
country_name


...

2019-10-22 12:33:25
country_name


127.0.0.1 - - [23/Oct/2019 00:00:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:00:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:00:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:00:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:00:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:00:39] "POST /_dash-update-component HTTP/1.1" 200 -


2019-10-22 12:41:05
country_name


127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 00:17:29] "POST /_dash-update-component HTTP/1.1" 200 -


2019-10-21 13:15:02
country_name
